<a href="https://colab.research.google.com/github/Aytsan11/ML/blob/main/Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install  pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy3
import json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from nltk.corpus import stopwords

# Инициализируем лемматизатор и список стоп-слов
morph = pymorphy3.MorphAnalyzer()
russian_stopwords = set(stopwords.words('russian'))

# Чтение данных из txt файла
with open('/content/anek.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Разбиваем текст на шутки по разделителю ***
jokes = [joke.strip() for joke in content.split('***') if joke.strip()]

def lemmatize_text(text):
    words = text.split()
    lemmas = []
    for word in words:
        p = morph.parse(word)[0]
        lemma = p.normal_form
        # Оставляем только алфавитные слова, не являющиеся стоп-словами
        if lemma.isalpha() and lemma not in russian_stopwords:
            lemmas.append(lemma)
    return lemmas

# Обрабатываем все шутки
processed_jokes = []
for joke in jokes:
    lemmas = lemmatize_text(joke)
    processed_jokes.append(lemmas)

# Пример вывода: лемматизированные слова из первых трёх шуток
for i, lemmas in enumerate(processed_jokes[:3], 1):
    print(f"Шутка {i}: {lemmas}")

Шутка 1: ['лошадь', 'мочь', 'смотреть', 'всё', 'твой', 'батя']
Шутка 2: ['никакой']
Шутка 3: ['вести', 'лапать']


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

processed_jokes_text = [' '.join(lemmas) for lemmas in processed_jokes]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_jokes_text)

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [29]:
# Параметры DBSCAN могут требовать подбора
dbscan = DBSCAN(eps=0.75, min_samples=60, metric='cosine')
labels = dbscan.fit_predict(X)

In [30]:
import numpy as np

unique_labels = np.unique(labels)
print(unique_labels)

[-1  0  1  2  3  4]


In [31]:
# Создаем DataFrame с заголовками и метками кластеров
df = pd.DataFrame({'joke': processed_jokes_text, 'class': labels})

# Выводим количество кластеров
unique_labels = set(labels)
n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
print(f'Количество кластеров: {n_clusters}')

# Анализируем кластеры
for label in unique_labels:
    if label == -1:
        continue  # Пропускаем шум
    cluster_titles = df[df['class'] == label]['joke'].head(3)
    print(f'\nКластер {label}:')
    for title in cluster_titles:
        print(f'- {title}')

Количество кластеров: 5

Кластер 0:
- сосаться ленин ленин горький
- штирлиц почувствовать подумать штирлиц
- дверь постучать удар десять тысяча подумать брюс

Кластер 1:
- называться отношение грубый водитель
- звать который
- называться обучать гей порно самый

Кластер 2:
- почему шутить это ещё жопа
- почему шутить это ещё
- муж непосредственный руководитель это свой плюс переспать это минус переспать это

Кластер 3:
- мужик пристать мужик разозлиться говорить твой мать вчера
- купить кружка знать чай
- пиво купить пить пенный

Кластер 4:
- забыть первый слово сын всё
- отец случайно проглотить папка
- приходить старший дочь отец младший кричать свой отец сын свой комната отец камень душа спасть домой мужик


In [32]:
df_1 = pd.DataFrame({'joke': jokes, 'class': labels})

In [33]:
df_1

,joke,class
0,Лошадь может смотреть на все 360 градусов. \nА...,-1
1,"— Дорогая, я... \n— Нет никаких ""я"" или ""ты"". ...",-1
2,"— Ты вел себя, как джентельмен. Спасибо, что н...",-1
3,В казарму новобранцев заходит лейтенант: \n— К...,-1
4,Поручик Ржевский проснулся и обнаружил между з...,-1
...,...,...
8231,"Мужик приходит к врачу:\n— Доктор, помогите, у...",-1
8232,Пришел грузин в автосалон машину покупать. Спр...,-1
8233,"Просыпается поручик Ржевский с бодуна, причем ...",-1
8234,"Приходит папа-людоед домой, приносит гроб с по...",-1


In [34]:
df_1['class'].value_counts()

,count
class,
-1,7430
1,268
0,181
3,175
4,101
2,81


In [37]:
df_2 = df_1[df_1["class"] != -1]

In [38]:
df_2['class'].value_counts()

,count
class,
1,268
0,181
3,175
4,101
2,81


In [39]:
df_2.to_csv('jokes.csv', index=False, encoding='utf-8')